In [143]:
import os
import numpy as np
import random
import cv2
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
'''
실행시키지 말 것 dataset zip파일 압축 푸는 코드
'''
# import zipfile
# DIV2K = zipfile.ZipFile('/home/lahj91/SR/DIV2K_valid_HR.zip')
# DIV2K.extractall('/home/lahj91/SR/DIV2K')
# DIV2K.close()

In [144]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./SR_training_datasets/BSDS200')#해당주소에서 data 이름 list로 추출
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("BSDS200_DataName.csv", index=False) #csv 파일로 저장

In [290]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_train_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_train_DataName.csv", index=False) #csv 파일로 저장

In [299]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_valid_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_valid_DataName.csv", index=False) #csv 파일로 저장

In [332]:
'''
부가적인 함수들 밑에는 img2tensor만 사용함
'''
def img2tensor(img, normalize=True):
    # handle gray image
    if len(img.shape)==2:
        img = img[:,:,np.newaxis]
    img = np.transpose(img, axes=(2,0,1)).astype(np.float32)
    if normalize:
        img = img / img.max()
    return torch.from_numpy(img)

def tensor2img(tensor):
    if len(tensor.shape)==2:
        img = tensor.cpu().numpy()
    elif len(tensor.shape)==3:
        img = np.transpose(tensor.cpu().numpy(), axes=(1,2,0))
    return img

def read_img_as_tensor(img_path, img_size=224, normalize=True):
    if isinstance(img_size, int):
        img_size = (img_size, img_size)
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img2tensor(img, normalize=normalize)
    return img

In [314]:
'''
torch DataLoader 작동 확인
'''
train_data = pd.read_csv('/home/lahj91/SR/DIV2K_train_DataName.csv') #csv파일 load
dataloader = dataset_SR(train_data) #dataset_SR class 불러오기
dataloader = DataLoader(dataloader, batch_size=3, shuffle=True) #torch datloader 정의

In [400]:
'''
dataset loader code
'''
class dataset_SR(Dataset):
    def __init__(self, df, 
                SR_mode = 96, 
                transforms=None, 
                flip=0.5, 
                rotation = [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_180, cv2.ROTATE_90_COUNTERCLOCKWISE],
                degradation = [cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_AREA, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4], 
                channel_wise_noise = True, 
                data='DIV2K',
                setting = "train"):
        super(dataset_SR, self).__init__()
        self.df = df
        self.SR_mode = SR_mode
        self.transforms = transforms
        self.flip = flip
        self.rotation = rotation
        self.data = data
        self.degradation = degradation
        self.channel_wise_noise = channel_wise_noise
        self.setting = setting
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.setting == "train":
            if self.data == 'BSDS200': #DIV2K로 일단 할 거기 때문에 무시
                view_path = '/home/lahj91/SR/SR_training_datasets/BSDS200'+'/'+self.df.iloc[idx]['name']
                origin = cv2.imread(view_path, cv2.IMREAD_UNCHANGED)
                p, q = random.randint(0, origin.shape[0]-self.SR_mode), random.randint(0, origin.shape[1]-self.SR_mode)
                origin = origin[p:p+self.SR_mode,  q:q+self.SR_mode] #random crop(96*96)으로 origin data 생성
                if random.random() < self.flip: #좌우반전
                    origin = cv2.flip(origin, 1)
                if self.rotation != False: #rotation
                    rd = random.random()
                    if rd < 3/4:
                        origin = cv2.rotate(origin, random.choice(self.rotation))
                    else:
                        origin = origin
                degraded = cv2.resize(origin, dsize=(int(self.SR_mode/2),int(self.SR_mode/2)), fx=0.5, fy=0.5, interpolation=random.choice(self.degradation)) 
                interpolated = cv2.resize(degraded, dsize=(self.SR_mode,self.SR_mode), fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC) #bicubic으로 degraded를 96*96 size 복원
                if self.transforms:
                    for transform in self.transforms:
                        origin = transform(origin)


            if self.data == 'DIV2K':
                view_path = '/home/lahj91/SR/DIV2K/DIV2K_train_HR'+'/'+self.df.iloc[idx]['name']
                origin = cv2.imread(view_path, cv2.IMREAD_UNCHANGED)
                p, q = random.randint(0, origin.shape[0]-self.SR_mode), random.randint(0, origin.shape[1]-self.SR_mode)
                origin = origin[p:p+self.SR_mode,  q:q+self.SR_mode] #random crop(96*96)으로 origin data 생성
                if self.channel_wise_noise: #channel-wise noise
                    pn = np.random.uniform(*[1 - 0.4, 1 + 0.4], size=(3))
                    origin = np.minimum(255., np.maximum(0., origin.transpose(2,0,1) * pn[:,np.newaxis,np.newaxis])).transpose(1,2,0)
                if random.random() < self.flip: #좌우반전
                    origin = cv2.flip(origin, 1)
                if self.rotation != False: #rotation
                    rd = random.random()
                    if rd < 3/4:
                        origin = cv2.rotate(origin, random.choice(self.rotation))
                    else:
                        origin = origin

                degraded = cv2.resize(origin, dsize=(int(self.SR_mode/2),int(self.SR_mode/2)), fx=0.5, fy=0.5, interpolation=random.choice(self.degradation)) #48*48로 random degradation
                interpolated = cv2.resize(degraded, dsize=(self.SR_mode,self.SR_mode), fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC) #bicubic으로 degraded를 96*96 size 복원
                if self.transforms:
                    for transform in self.transforms:
                        origin = transform(origin)
                        
        elif self.setting == "val":
            if self.data == 'DIV2K':
                view_path = '/home/lahj91/SR/DIV2K/DIV2K_valid_HR'+'/'+self.df.iloc[idx]['name']
                origin = cv2.imread(view_path, cv2.IMREAD_UNCHANGED)
                p, q = random.randint(0, origin.shape[0]-self.SR_mode), random.randint(0, origin.shape[1]-self.SR_mode)
                origin = origin[p:p+self.SR_mode,  q:q+self.SR_mode] #random crop(96*96)으로 origin data 생성
                degraded = cv2.resize(origin, dsize=(int(self.SR_mode/2),int(self.SR_mode/2)), fx=0.5, fy=0.5, interpolation=random.choice(self.degradation)) #48*48로 random degradation
                interpolated = cv2.resize(degraded, dsize=(self.SR_mode,self.SR_mode), fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC) #bicubic으로 degraded를 96*96 size 복원
                if self.transforms:
                    for transform in self.transforms:
                        origin = transform(origin)

        return torch.from_numpy(origin.transpose(2,0,1)), torch.from_numpy(degraded.transpose(2,0,1)), torch.from_numpy(interpolated.transpose(2,0,1))


In [402]:
train_data = pd.read_csv('/home/lahj91/SR/DIV2K_train_DataName.csv') #csv파일 load
dataloader = dataset_SR(train_data) #dataset_SR class 불러오기
origin, degraded, interpolated = dataloader.__getitem__(2) #test code, 저장하여 확인(cv2.imshow가 안됨;;; 쓰지말 것 kernel 터짐)

'''
아래는 잘 뽑히고 있는지 시각화(저장됨)하는 코드
'''
cv2.imwrite('/home/lahj91/SR/origin.png',tensor2img(origin))
cv2.imwrite('/home/lahj91/SR/degraded.png',tensor2img(degraded))
cv2.imwrite('/home/lahj91/SR/interpolated.png',tensor2img(interpolated))

True

In [399]:
train_data.iloc[2] #몇번 data가 뽑혔는지 확인하고 싶으면 해당 data가서 보기

name    0027.png
Name: 2, dtype: object

In [355]:
train_data = pd.read_csv('/home/lahj91/SR/DIV2K_valid_DataName.csv') #csv파일 load
dataloader = dataset_SR(train_data, setting="val") #dataset_SR class 불러오기
origin, degraded, interpolated = dataloader.__getitem__(2) #test code, 저장하여 확인(cv2.imshow가 안됨;;; 쓰지말 것 kernel 터짐)
cv2.imwrite('/home/lahj91/SR/origin.png',tensor2img(origin))
cv2.imwrite('/home/lahj91/SR/degraded.png',tensor2img(degraded))
cv2.imwrite('/home/lahj91/SR/interpolated.png',tensor2img(interpolated))

True